# RNA Foundation Model

## 핵심 설계 원칙

### 1. 불변 물리 특징 vs 학습 가능 임베딩

```
원자 표현 = [불변 물리 특징] + [학습 가능 임베딩]
             (freeze)           (update)

물리 특징 (14-dim):           학습 임베딩:
- 원자번호 (불변)              - 문맥 정보 (학습)
- VdW 반경 (불변)              - 상호작용 정보 (학습)
- 전기음성도 (불변)            - 구조적 역할 (학습)
- H-bond donor/acceptor (불변)
```

### 2. 정보 흐름

```
물리 특징 (불변) ──┬──> 물리 엔진 (H-bond, Clash 계산)
                  │
                  ├──> Attention Query/Key (물리적 호환성)
                  │
학습 임베딩 ──────┴──> GNN, Attention으로 업데이트
                           ↓
                      R, T 예측
```

In [1]:
# 환경 설정
!pip install torch torch-geometric rdkit matplotlib networkx -q

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, HeteroData
from torch_geometric.nn import TransformerConv, GATConv
import math
import numpy as np
from typing import Dict, List, Tuple, Optional
from enum import IntEnum
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx

print(f"PyTorch: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 63.0 MB/s eta 0:00:00
PyTorch: 2.9.0+cu126
Device: cuda


## 1. 물리 상수 정의 (불변)

In [2]:
"""
물리 상수 - 이 값들은 절대 학습되지 않음
"""

ATOM_PHYSICS = {
    1:  {'symbol': 'H',  'vdw_radius': 1.20, 'mass': 1.008,  'electronegativity': 2.20},
    6:  {'symbol': 'C',  'vdw_radius': 1.70, 'mass': 12.011, 'electronegativity': 2.55},
    7:  {'symbol': 'N',  'vdw_radius': 1.55, 'mass': 14.007, 'electronegativity': 3.04},
    8:  {'symbol': 'O',  'vdw_radius': 1.52, 'mass': 15.999, 'electronegativity': 3.44},
    15: {'symbol': 'P',  'vdw_radius': 1.80, 'mass': 30.974, 'electronegativity': 2.19},
}

HBOND_GEOMETRY = {
    'distance_range': (2.4, 3.5),
    'optimal_distance': 2.8,
}

BASE_SMILES = {
    'A': 'Nc1ncnc2[nH]cnc12',
    'G': 'Nc1nc2[nH]cnc2c(=O)[nH]1',
    'U': 'O=c1cc[nH]c(=O)[nH]1',
    'C': 'Nc1cc[nH]c(=O)n1',
}

class SugarPucker(IntEnum):
    C2_ENDO = 0
    C3_ENDO = 1

def vdw_volume(r): return (4/3) * math.pi * (r ** 3)

def get_initial_template(base):
    mol = Chem.MolFromSmiles(BASE_SMILES[base])
    try:
        AllChem.Compute2DCoords(mol)
        conf = mol.GetConformer()
        return torch.tensor([[conf.GetAtomPosition(i).x, conf.GetAtomPosition(i).y, 0.0]
                             for i in range(mol.GetNumAtoms())], dtype=torch.float)
    except:
        n = mol.GetNumAtoms()
        angles = torch.linspace(0, 2*math.pi, n+1)[:-1]
        return torch.stack([torch.cos(angles), torch.sin(angles), torch.zeros(n)], dim=1) * 1.5

print("물리 상수 정의 완료 (불변)")

물리 상수 정의 완료 (불변)


## 2. 원자 특징 분류

**중요:** 물리적 특징과 학습 가능 특징을 명확히 분리

In [3]:
"""
원자 특징 분류:

1. 불변 물리 특징 (INVARIANT_PHYSICS_DIM = 10)
   - 원자번호, 질량, VdW 반경, 전기음성도
   - H-bond donor/acceptor 여부
   - 이 값들은 물리 법칙에 의해 결정되며, 절대 학습되지 않음

2. 구조적 특징 (STRUCTURAL_DIM = 4)
   - 혼성 오비탈, 방향족성, 고리 포함, 차수
   - 분자 구조에서 파생되지만, 문맥에 따라 해석이 달라질 수 있음
"""

INVARIANT_PHYSICS_DIM = 10  # 불변 물리 특징 차원
STRUCTURAL_DIM = 4          # 구조적 특징 차원
TOTAL_ATOM_DIM = INVARIANT_PHYSICS_DIM + STRUCTURAL_DIM  # 14


def get_atom_features(atom):
    """
    원자 특징 추출

    Returns:
        physics_features: [10] 불변 물리 특징
        structural_features: [4] 구조적 특징
    """
    atomic_num = atom.GetAtomicNum()
    props = ATOM_PHYSICS.get(atomic_num, ATOM_PHYSICS[6])

    num_hs = atom.GetTotalNumHs()
    is_electronegative = atomic_num in [7, 8]  # N, O

    # 불변 물리 특징 (10-dim) - 절대 학습되지 않음
    physics_features = [
        float(atomic_num),                          # 0: 원자번호
        props['mass'],                              # 1: 질량
        props['vdw_radius'],                        # 2: VdW 반경
        vdw_volume(props['vdw_radius']),            # 3: VdW 부피
        props['electronegativity'],                 # 4: 전기음성도
        float(is_electronegative),                  # 5: 전기음성 원자 여부
        float(is_electronegative and num_hs > 0),   # 6: H-bond donor
        float(is_electronegative),                  # 7: H-bond acceptor
        float(num_hs),                              # 8: 수소 개수
        float(atom.GetFormalCharge()),              # 9: 형식 전하
    ]

    # 구조적 특징 (4-dim) - 분자 구조에서 파생
    hyb = {Chem.rdchem.HybridizationType.SP: 1, Chem.rdchem.HybridizationType.SP2: 2,
           Chem.rdchem.HybridizationType.SP3: 3}.get(atom.GetHybridization(), 0)

    structural_features = [
        float(hyb),                     # 0: 혼성 오비탈
        float(atom.GetIsAromatic()),    # 1: 방향족성
        float(atom.IsInRing()),         # 2: 고리 포함
        float(atom.GetDegree()),        # 3: 결합 차수
    ]

    return physics_features, structural_features


def get_sugar_features(pucker=SugarPucker.C3_ENDO):
    """당 특징 (8차원)"""
    return [1.0, float(pucker == SugarPucker.C2_ENDO), float(pucker == SugarPucker.C3_ENDO),
            1.0, 134.13, 5*vdw_volume(1.70)+4*vdw_volume(1.52), 5.0, 4.0]


def get_phosphate_features():
    """인산 특징 (8차원)"""
    return [2.0, -1.0, 94.97, vdw_volume(1.80)+4*vdw_volume(1.52), 1.0, 4.0, 1.80, 2.19]


print(f"원자 특징: 불변 물리({INVARIANT_PHYSICS_DIM}) + 구조적({STRUCTURAL_DIM}) = {TOTAL_ATOM_DIM}차원")

원자 특징: 불변 물리(10) + 구조적(4) = 14차원


## 3. 이질적 RNA 그래프 (물리/구조 분리)

In [ ]:
"""
이질적 RNA 그래프 - 물리 특징과 구조 특징 분리 저장
"""

class HeterogeneousRNAGraph:
    def __init__(self, sequence: str, device: torch.device = None):
        self.sequence = sequence.upper()
        self.num_nucleotides = len(sequence)
        self.device = device or torch.device('cpu')
        self._build()

    def _find_glycosidic_atom(self, mol, base):
        for atom in mol.GetAtoms():
            if atom.GetSymbol() == 'N':
                if base in ['A', 'G']:
                    if [n.GetSymbol() for n in atom.GetNeighbors()].count('C') >= 2:
                        return atom.GetIdx()
                else:
                    if atom.IsInRing():
                        return atom.GetIdx()
        return 0

    def _build(self):
        # 분리된 특징 저장
        physics_features = []      # 불변 물리 특징 (10-dim)
        structural_features = []   # 구조적 특징 (4-dim)
        sugar_features = []
        phosphate_features = []

        bond_edges = [[], []]
        glycosidic_edges, backbone_edges, phos_sugar_edges = [[],[]], [[],[]], [[],[]]

        self.atom_to_nucleotide = []
        self.atom_symbols = []
        self.base_types = []
        self.glycosidic_atoms = []
        self.templates = []
        self.atom_to_template_idx = []
        self.nuc_atom_ranges = []

        atom_offset = 0

        for nuc_idx, base in enumerate(self.sequence):
            mol = Chem.MolFromSmiles(BASE_SMILES[base])
            template = get_initial_template(base)
            self.templates.append(template)

            glyc_local = self._find_glycosidic_atom(mol, base)
            glyc_global = atom_offset + glyc_local
            self.glycosidic_atoms.append(glyc_global)

            start_idx = atom_offset

            for local_idx, atom in enumerate(mol.GetAtoms()):
                phys, struct = get_atom_features(atom)
                physics_features.append(phys)
                structural_features.append(struct)

                self.atom_to_nucleotide.append(nuc_idx)
                self.atom_symbols.append(atom.GetSymbol())
                self.base_types.append(base)
                self.atom_to_template_idx.append(local_idx)

            for bond in mol.GetBonds():
                i, j = atom_offset + bond.GetBeginAtomIdx(), atom_offset + bond.GetEndAtomIdx()
                bond_edges[0].extend([i, j])
                bond_edges[1].extend([j, i])

            atom_offset += mol.GetNumAtoms()
            self.nuc_atom_ranges.append((start_idx, atom_offset))

            sugar_features.append(get_sugar_features())
            phosphate_features.append(get_phosphate_features())

            glycosidic_edges[0].append(nuc_idx)
            glycosidic_edges[1].append(glyc_global)
            phos_sugar_edges[0].append(nuc_idx)
            phos_sugar_edges[1].append(nuc_idx)

            if nuc_idx > 0:
                backbone_edges[0].append(nuc_idx - 1)
                backbone_edges[1].append(nuc_idx)

        self.num_base_atoms = len(physics_features)
        self.num_sugars = len(sugar_features)
        self.num_phosphates = len(phosphate_features)

        # 분리 저장: 물리 특징 (불변) vs 구조 특징
        self.physics_x = torch.tensor(physics_features, dtype=torch.float, device=self.device)
        self.structural_x = torch.tensor(structural_features, dtype=torch.float, device=self.device)
        self.base_atom_x = torch.cat([self.physics_x, self.structural_x], dim=-1)

        self.sugar_x = torch.tensor(sugar_features, dtype=torch.float, device=self.device)
        self.phosphate_x = torch.tensor(phosphate_features, dtype=torch.float, device=self.device)

        def to_edge(e): return torch.tensor(e, dtype=torch.long, device=self.device) if e[0] else torch.zeros(2, 0, dtype=torch.long, device=self.device)

        self.bond_edge_index = to_edge(bond_edges)
        self.glycosidic_edge_index = to_edge(glycosidic_edges)
        self.backbone_edge_index = to_edge(backbone_edges)
        self.phos_sugar_edge_index = to_edge(phos_sugar_edges)

        self.atom_to_nucleotide = torch.tensor(self.atom_to_nucleotide, dtype=torch.long, device=self.device)
        self.atom_to_template_idx = torch.tensor(self.atom_to_template_idx, dtype=torch.long, device=self.device)

        # 템플릿도 device로 이동
        self.templates = [t.to(self.device) for t in self.templates]

        # 물리 엔진용 특징 (불변 물리 특징에서 추출)
        self.vdw_radii = self.physics_x[:, 2]   # VdW 반경
        self.is_donor = self.physics_x[:, 6]    # H-bond donor
        self.is_acceptor = self.physics_x[:, 7] # H-bond acceptor

    def to(self, device: torch.device):
        """Device 이동"""
        self.device = device
        self.physics_x = self.physics_x.to(device)
        self.structural_x = self.structural_x.to(device)
        self.base_atom_x = self.base_atom_x.to(device)
        self.sugar_x = self.sugar_x.to(device)
        self.phosphate_x = self.phosphate_x.to(device)
        self.bond_edge_index = self.bond_edge_index.to(device)
        self.glycosidic_edge_index = self.glycosidic_edge_index.to(device)
        self.backbone_edge_index = self.backbone_edge_index.to(device)
        self.phos_sugar_edge_index = self.phos_sugar_edge_index.to(device)
        self.atom_to_nucleotide = self.atom_to_nucleotide.to(device)
        self.atom_to_template_idx = self.atom_to_template_idx.to(device)
        self.templates = [t.to(device) for t in self.templates]
        self.vdw_radii = self.vdw_radii.to(device)
        self.is_donor = self.is_donor.to(device)
        self.is_acceptor = self.is_acceptor.to(device)
        return self

    def to_hetero_data(self):
        data = HeteroData()

        # 물리 특징과 구조 특징 분리 저장
        data['base_atom'].physics_x = self.physics_x        # 불변
        data['base_atom'].structural_x = self.structural_x  # 구조
        data['base_atom'].x = self.base_atom_x              # 전체 (참조용)

        data['sugar'].x = self.sugar_x
        data['phosphate'].x = self.phosphate_x

        if self.bond_edge_index.numel() > 0:
            data['base_atom', 'bond', 'base_atom'].edge_index = self.bond_edge_index
        if self.glycosidic_edge_index.numel() > 0:
            data['sugar', 'glycosidic', 'base_atom'].edge_index = self.glycosidic_edge_index
            data['base_atom', 'rev_glycosidic', 'sugar'].edge_index = self.glycosidic_edge_index.flip(0)
        if self.backbone_edge_index.numel() > 0:
            data['sugar', 'backbone', 'sugar'].edge_index = self.backbone_edge_index
        if self.phos_sugar_edge_index.numel() > 0:
            data['phosphate', 'connects', 'sugar'].edge_index = self.phos_sugar_edge_index

        data.num_nucleotides = self.num_nucleotides
        data.sequence = self.sequence
        return data

    def apply_transforms(self, quaternions, translations):
        coords = torch.zeros(self.num_base_atoms, 3, device=self.device)
        for nuc_idx in range(self.num_nucleotides):
            mask = self.atom_to_nucleotide == nuc_idx
            atom_indices = mask.nonzero(as_tuple=True)[0]
            template_indices = self.atom_to_template_idx[atom_indices]
            local = self.templates[nuc_idx][template_indices]
            coords[atom_indices] = apply_rigid_transform(local, quaternions[nuc_idx], translations[nuc_idx])
        return coords


graph = HeterogeneousRNAGraph("GCAU")
print(f"그래프: base_atom={graph.num_base_atoms}")
print(f"  - 물리 특징: {graph.physics_x.shape}")
print(f"  - 구조 특징: {graph.structural_x.shape}")

그래프: base_atom=37
  - 물리 특징: torch.Size([37, 10])
  - 구조 특징: torch.Size([37, 4])


## 4. SE(3) Rigid Transform

In [5]:
def quaternion_to_rotation_matrix(q):
    q = F.normalize(q, dim=-1)
    if q.dim() == 1: q = q.unsqueeze(0)
    w, x, y, z = q[..., 0], q[..., 1], q[..., 2], q[..., 3]
    return torch.stack([
        torch.stack([1-2*y*y-2*z*z, 2*x*y-2*z*w, 2*x*z+2*y*w], dim=-1),
        torch.stack([2*x*y+2*z*w, 1-2*x*x-2*z*z, 2*y*z-2*x*w], dim=-1),
        torch.stack([2*x*z-2*y*w, 2*y*z+2*x*w, 1-2*x*x-2*y*y], dim=-1),
    ], dim=-2)

def apply_rigid_transform(coords, rotation, translation):
    if rotation.shape[-1] == 4:
        R = quaternion_to_rotation_matrix(rotation)
        if R.dim() == 3: R = R.squeeze(0)
    else:
        R = rotation
    return coords @ R.T + translation

print("SE(3) Transform 정의 완료")

SE(3) Transform 정의 완료


## 5. 물리 엔진 (불변 특징 사용)

In [6]:
"""
물리 엔진 - 불변 물리 특징만 사용

중요: VdW 반경, H-bond donor/acceptor는 불변 물리 특징에서 가져옴
      학습된 임베딩이 아니라 실제 물리 상수 사용
"""

class DifferentiablePhysicsEngine:
    def __init__(self):
        self.hbond_geo = HBOND_GEOMETRY

    def compute_clash_loss(self, coords, vdw_radii, nuc_ids, eps=1e-8):
        """VdW 충돌 계산 - vdw_radii는 불변 물리 특징"""
        N = coords.size(0)
        dist = torch.cdist(coords, coords) + eps
        min_allowed = (vdw_radii.unsqueeze(0) + vdw_radii.unsqueeze(1)) * 0.75
        same_nuc = nuc_ids.unsqueeze(0) == nuc_ids.unsqueeze(1)
        exclude = same_nuc | torch.eye(N, dtype=torch.bool, device=coords.device)
        pen = F.relu(min_allowed - dist).masked_fill(exclude, 0.0)
        return (pen ** 2).sum() / 2, (pen > 0.1).sum() / 2

    def detect_hbonds(self, coords, is_donor, is_acceptor, nuc_ids, eps=1e-8):
        """H-bond 검출 - is_donor, is_acceptor는 불변 물리 특징"""
        dist = torch.cdist(coords, coords) + eps

        # 물리적으로 가능한 쌍만 (donor ↔ acceptor)
        valid = ((is_donor.unsqueeze(1) * is_acceptor.unsqueeze(0)) +
                 (is_acceptor.unsqueeze(1) * is_donor.unsqueeze(0))).clamp(max=1.0)
        valid = valid * (nuc_ids.unsqueeze(0) != nuc_ids.unsqueeze(1)).float()

        d_opt, (d_min, d_max) = self.hbond_geo['optimal_distance'], self.hbond_geo['distance_range']
        score = torch.exp(-((dist - d_opt) / ((d_max - d_opt) / 2)) ** 2)
        score = score * ((dist >= d_min) & (dist <= d_max)).float()
        strength = score * valid
        return strength, -5.0 * strength.sum() / 2, (strength > 0.5).sum() / 2

    def evaluate(self, coords, vdw_radii, is_donor, is_acceptor, nuc_ids):
        clash_loss, clash_count = self.compute_clash_loss(coords, vdw_radii, nuc_ids)
        hbond_strength, hbond_energy, hbond_count = self.detect_hbonds(coords, is_donor, is_acceptor, nuc_ids)
        return {
            'clash_loss': clash_loss, 'clash_count': clash_count,
            'hbond_strength': hbond_strength, 'hbond_energy': hbond_energy, 'hbond_count': hbond_count,
        }

print("물리 엔진 정의 완료 (불변 특징 사용)")

물리 엔진 정의 완료 (불변 특징 사용)


## 6. 핵심: 불변 특징 + 학습 임베딩 분리

### 설계 원칙

```
원자 표현 = [불변 물리 특징] concat [학습 임베딩]
              (freeze)              (update)

GNN/Attention:
- Query/Key: 불변 물리 특징 포함 (물리적 호환성 판단)
- Value/Update: 학습 임베딩만 업데이트
- 최종 출력: 불변 특징 + 업데이트된 학습 임베딩
```

In [7]:
"""
불변 특징 + 학습 임베딩 분리 구조
"""


class AtomEmbedding(nn.Module):
    """
    원자 임베딩 - 불변 특징과 학습 특징 분리

    구조:
    - physics_x (10-dim): 불변, 학습 X
    - structural_x (4-dim): 구조적 특징
    - learnable_emb (hidden-dim): 학습 가능한 문맥 임베딩

    출력:
    - full_repr = concat(physics_x, learnable_emb)  # 물리 특징 항상 유지
    """

    def __init__(self, hidden_dim: int):
        super().__init__()
        self.hidden_dim = hidden_dim

        # 구조적 특징 → 학습 임베딩 초기화
        # (물리 특징은 직접 변환하지 않음, 참조만)
        self.structural_encoder = nn.Sequential(
            nn.Linear(STRUCTURAL_DIM, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.SiLU(),
        )

        # 물리 특징을 "참조"하여 학습 임베딩에 영향
        # (물리 특징 자체는 변환 X, bias로만 사용)
        self.physics_bias = nn.Linear(INVARIANT_PHYSICS_DIM, hidden_dim, bias=False)

    def forward(self, physics_x: torch.Tensor, structural_x: torch.Tensor):
        """
        Args:
            physics_x: [N, 10] 불변 물리 특징
            structural_x: [N, 4] 구조적 특징

        Returns:
            physics_x: [N, 10] 불변 (그대로 반환)
            learnable_emb: [N, hidden] 학습 가능한 임베딩
        """
        # 구조적 특징 → 학습 임베딩
        learnable_emb = self.structural_encoder(structural_x)

        # 물리 특징은 bias로만 영향 (값 자체는 변환 X)
        learnable_emb = learnable_emb + self.physics_bias(physics_x)

        # 불변 물리 특징은 그대로 반환
        return physics_x, learnable_emb


class PhysicsAwareGNN(nn.Module):
    """
    물리 특징을 인식하는 GNN

    핵심:
    - Query/Key 계산: 불변 물리 특징 + 학습 임베딩 모두 사용
    - Value 업데이트: 학습 임베딩만 업데이트
    - 출력: 불변 물리 특징은 그대로 유지
    """

    def __init__(self, hidden_dim: int):
        super().__init__()

        # Attention을 위한 projection
        # 입력: 물리 특징(10) + 학습 임베딩(hidden)
        total_dim = INVARIANT_PHYSICS_DIM + hidden_dim

        self.q_proj = nn.Linear(total_dim, hidden_dim)
        self.k_proj = nn.Linear(total_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)  # 학습 임베딩만

        self.out_proj = nn.Linear(hidden_dim, hidden_dim)
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(
        self,
        physics_x: torch.Tensor,     # [N, 10] 불변
        learnable_emb: torch.Tensor, # [N, hidden] 학습 가능
        edge_index: torch.Tensor,
    ) -> torch.Tensor:
        """
        Returns:
            learnable_emb_updated: [N, hidden] 업데이트된 학습 임베딩
            (physics_x는 변경 없이 호출자가 유지)
        """
        if edge_index.numel() == 0:
            return learnable_emb

        # 전체 표현 (물리 + 학습) → Query/Key 계산
        full_repr = torch.cat([physics_x, learnable_emb], dim=-1)  # [N, 10+hidden]

        Q = self.q_proj(full_repr)  # [N, hidden]
        K = self.k_proj(full_repr)  # [N, hidden]
        V = self.v_proj(learnable_emb)  # [N, hidden] - 학습 임베딩만!

        # 에지 기반 attention
        src, dst = edge_index

        # Attention score: Q[dst] · K[src]
        attn_scores = (Q[dst] * K[src]).sum(dim=-1) / math.sqrt(Q.size(-1))

        # Softmax per destination node
        attn_weights = torch.zeros(learnable_emb.size(0), device=learnable_emb.device)
        attn_weights.scatter_add_(0, dst, torch.exp(attn_scores))
        attn_probs = torch.exp(attn_scores) / (attn_weights[dst] + 1e-8)

        # Message passing: V[src] * attn_probs → aggregate to dst
        msg = V[src] * attn_probs.unsqueeze(-1)
        out = torch.zeros_like(learnable_emb)
        out.scatter_add_(0, dst.unsqueeze(-1).expand_as(msg), msg)

        out = self.out_proj(out)

        # Residual + Norm (학습 임베딩만 업데이트)
        learnable_emb_updated = self.norm(learnable_emb + out)

        return learnable_emb_updated


print("물리 인식 GNN 정의 완료")

물리 인식 GNN 정의 완료


In [8]:
"""
물리 기반 마스킹 Cross-Attention

핵심 최적화:
- 물리적으로 상호작용 불가능한 원자 쌍은 attention 계산에서 제외
- H-bond: donor ↔ acceptor 쌍만
- π-π Stacking: aromatic ↔ aromatic 쌍만
- 이외의 쌍: 마스킹 (attention = 0)

효과:
- 계산 효율: O(N²) → O(가능한 쌍만)
- 물리적 의미: 실제 상호작용 가능한 쌍에만 집중
- 노이즈 감소: 불필요한 attention이 학습을 방해하지 않음
"""


def compute_physics_interaction_mask(
    physics_x: torch.Tensor,      # [N, 10] 물리 특징
    structural_x: torch.Tensor,   # [N, 4] 구조 특징 (aromatic 포함)
    atom_to_nuc: torch.Tensor,    # [N] 뉴클레오타이드 소속
) -> torch.Tensor:
    """
    물리적으로 상호작용 가능한 원자 쌍 마스크 계산

    Returns:
        valid_mask: [N, N] True = attention 계산 대상
    """
    N = physics_x.size(0)

    # 물리 특징에서 추출
    is_donor = physics_x[:, 6]      # H-bond donor
    is_acceptor = physics_x[:, 7]   # H-bond acceptor

    # 구조 특징에서 추출
    is_aromatic = structural_x[:, 1]  # 방향족성

    # 1. H-bond 가능 쌍: donor ↔ acceptor
    hbond_mask = (
        (is_donor.unsqueeze(1) * is_acceptor.unsqueeze(0)) +  # i=donor, j=acceptor
        (is_acceptor.unsqueeze(1) * is_donor.unsqueeze(0))    # i=acceptor, j=donor
    ).clamp(max=1.0) > 0

    # 2. π-π Stacking 가능 쌍: aromatic ↔ aromatic
    stacking_mask = (
        is_aromatic.unsqueeze(1) * is_aromatic.unsqueeze(0)
    ) > 0

    # 3. 다른 뉴클레오타이드 간만 (서브그래프 '간' attention)
    diff_nuc_mask = atom_to_nuc.unsqueeze(0) != atom_to_nuc.unsqueeze(1)

    # 최종: (H-bond 가능 OR Stacking 가능) AND 다른 뉴클레오타이드
    valid_mask = (hbond_mask | stacking_mask) & diff_nuc_mask

    return valid_mask


class PhysicsMaskedCrossAttention(nn.Module):
    """
    물리 기반 마스킹 Cross-Attention

    핵심:
    1. 물리적으로 불가능한 쌍은 사전에 마스킹 (attention 계산 X)
    2. Query/Key: 물리 + 학습 임베딩 사용
    3. Value: 학습 임베딩만 업데이트

    마스킹 기준:
    - H-bond: donor ↔ acceptor 쌍만 ✓
    - Stacking: aromatic ↔ aromatic 쌍만 ✓
    - 이외: 마스킹 (C ↔ C 등은 attention = 0)
    """

    def __init__(self, hidden_dim: int, num_heads: int = 4):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads

        total_dim = INVARIANT_PHYSICS_DIM + hidden_dim

        self.q_proj = nn.Linear(total_dim, hidden_dim)
        self.k_proj = nn.Linear(total_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)

        self.out_proj = nn.Linear(hidden_dim, hidden_dim)
        self.norm = nn.LayerNorm(hidden_dim)
        self.scale = math.sqrt(self.head_dim)

    def forward(
        self,
        physics_x: torch.Tensor,       # [N, 10] 불변 물리 특징
        structural_x: torch.Tensor,    # [N, 4] 구조 특징
        learnable_emb: torch.Tensor,   # [N, hidden] 학습 임베딩
        atom_to_nuc: torch.Tensor,     # [N] 뉴클레오타이드 소속
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Returns:
            learnable_emb_updated: [N, hidden]
            atom_pair_scores: [N, N] attention scores
            physics_mask: [N, N] 물리적으로 가능한 쌍 마스크
        """
        N = physics_x.size(0)

        # 1. 물리 기반 마스크 계산 (사전에!)
        physics_mask = compute_physics_interaction_mask(
            physics_x, structural_x, atom_to_nuc
        )  # [N, N]

        # 2. Q, K, V 계산
        full_repr = torch.cat([physics_x, learnable_emb], dim=-1)

        Q = self.q_proj(full_repr).view(N, self.num_heads, self.head_dim)
        K = self.k_proj(full_repr).view(N, self.num_heads, self.head_dim)
        V = self.v_proj(learnable_emb).view(N, self.num_heads, self.head_dim)

        # 3. Attention scores (물리적으로 가능한 쌍만!)
        attn_scores = torch.einsum('ihd,jhd->ijh', Q, K) / self.scale

        # 물리 마스크 적용: 불가능한 쌍은 -inf
        invalid_mask = ~physics_mask  # 불가능한 쌍
        attn_scores = attn_scores.masked_fill(invalid_mask.unsqueeze(-1), float('-inf'))

        # 4. Softmax (가능한 쌍에 대해서만)
        attn_probs = F.softmax(attn_scores, dim=1)
        attn_probs = attn_probs.masked_fill(invalid_mask.unsqueeze(-1), 0.0)

        # NaN 방지: 모든 쌍이 마스킹된 원자는 attention = 0
        attn_probs = torch.nan_to_num(attn_probs, nan=0.0)

        # 5. Value aggregation
        out = torch.einsum('ijh,jhd->ihd', attn_probs, V)
        out = out.reshape(N, self.hidden_dim)
        out = self.out_proj(out)

        # 6. Residual + Norm
        learnable_emb_updated = self.norm(learnable_emb + out)

        # 7. Pair scores (대칭화)
        atom_pair_scores = attn_probs.mean(dim=-1)
        atom_pair_scores = (atom_pair_scores + atom_pair_scores.T) / 2

        return learnable_emb_updated, atom_pair_scores, physics_mask


print("물리 마스킹 Cross-Attention 정의 완료")

물리 마스킹 Cross-Attention 정의 완료


## 7. Transform 예측기

In [9]:
"""
원자 → 뉴클레오타이드 Transform 예측
"""


class AtomToNucleotideTransform(nn.Module):
    def __init__(self, hidden_dim: int):
        super().__init__()

        self.sugar_enc = nn.Linear(8, hidden_dim)
        self.phos_enc = nn.Linear(8, hidden_dim)

        # Attention pooling (물리 + 학습 임베딩 모두 사용)
        total_dim = INVARIANT_PHYSICS_DIM + hidden_dim
        self.atom_pool_attn = nn.Sequential(
            nn.Linear(total_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1),
        )

        self.nuc_combine = nn.Sequential(
            nn.Linear(hidden_dim * 3, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.SiLU(),
        )

        self.rotation_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 4),
        )
        self.translation_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 3),
        )

        self._init_weights()

    def _init_weights(self):
        nn.init.zeros_(self.rotation_head[-1].weight)
        self.rotation_head[-1].bias.data = torch.tensor([1., 0., 0., 0.])
        nn.init.zeros_(self.translation_head[-1].weight)
        nn.init.zeros_(self.translation_head[-1].bias)

    def forward(
        self,
        physics_x: torch.Tensor,
        learnable_emb: torch.Tensor,
        atom_to_nuc: torch.Tensor,
        sugar_x: torch.Tensor,
        phos_x: torch.Tensor,
        num_nucleotides: int,
    ):
        h_sugar = self.sugar_enc(sugar_x)
        h_phos = self.phos_enc(phos_x)

        # 전체 표현 (물리 + 학습)
        full_repr = torch.cat([physics_x, learnable_emb], dim=-1)

        nuc_from_atoms = []
        for nuc_idx in range(num_nucleotides):
            mask = atom_to_nuc == nuc_idx
            h_atoms = full_repr[mask]
            learnable_atoms = learnable_emb[mask]

            attn = self.atom_pool_attn(h_atoms)
            attn_weights = F.softmax(attn, dim=0)
            pooled = (attn_weights * learnable_atoms).sum(dim=0)  # 학습 임베딩만 pooling
            nuc_from_atoms.append(pooled)

        nuc_from_atoms = torch.stack(nuc_from_atoms)

        nuc_emb = self.nuc_combine(torch.cat([nuc_from_atoms, h_sugar, h_phos], dim=-1))

        quaternions = F.normalize(self.rotation_head(nuc_emb), dim=-1)
        translations = self.translation_head(nuc_emb)

        return quaternions, translations, nuc_emb


print("Transform 예측기 정의 완료")

Transform 예측기 정의 완료


## 8. 통합 모델

In [ ]:
"""
통합 모델: 물리 마스킹 + 불변 특징 분리
"""


class PhysicsMaskedRNAModel(nn.Module):
    """
    물리 마스킹 기반 RNA 구조 예측 모델

    핵심 원칙:
    1. 물리적으로 불가능한 쌍은 attention에서 제외
       - H-bond: donor ↔ acceptor 만
       - Stacking: aromatic ↔ aromatic 만
       - 이외: 마스킹 (C ↔ C 등 제외)

    2. 불변 물리 특징 (10-dim): 절대 학습되지 않음

    3. 학습 임베딩 (hidden-dim): GNN, Cross-Attention으로 업데이트
    """

    def __init__(
        self,
        hidden_dim: int = 128,
        num_gnn_layers: int = 2,
        num_cross_attn_layers: int = 4,
        num_heads: int = 4,
    ):
        super().__init__()

        # 1. 원자 임베딩
        self.atom_embedding = AtomEmbedding(hidden_dim)

        # 2. 서브그래프 내 GNN
        self.intra_gnns = nn.ModuleList([
            PhysicsAwareGNN(hidden_dim) for _ in range(num_gnn_layers)
        ])

        # 3. 서브그래프 간 Cross-Attention (물리 마스킹!)
        self.cross_attns = nn.ModuleList([
            PhysicsMaskedCrossAttention(hidden_dim, num_heads)
            for _ in range(num_cross_attn_layers)
        ])

        # 4. Transform 예측
        self.transform_predictor = AtomToNucleotideTransform(hidden_dim)

        # 5. 물리 엔진
        self.physics = DifferentiablePhysicsEngine()

    def forward(self, graph: HeterogeneousRNAGraph):
        data = graph.to_hetero_data()

        # 불변 특징
        physics_x = data['base_atom'].physics_x      # [N, 10]
        structural_x = data['base_atom'].structural_x  # [N, 4]

        # 1. 초기 임베딩
        physics_x, learnable_emb = self.atom_embedding(physics_x, structural_x)

        # 2. 서브그래프 내 GNN
        bond_edge_index = data['base_atom', 'bond', 'base_atom'].edge_index \
            if ('base_atom', 'bond', 'base_atom') in data.edge_types else torch.zeros(2, 0, dtype=torch.long)

        for gnn in self.intra_gnns:
            learnable_emb = gnn(physics_x, learnable_emb, bond_edge_index)

        # 3. 서브그래프 간 Cross-Attention (물리 마스킹!)
        atom_pair_scores_list = []
        physics_mask = None

        for cross_attn in self.cross_attns:
            learnable_emb, atom_pair_scores, physics_mask = cross_attn(
                physics_x, structural_x, learnable_emb, graph.atom_to_nucleotide
            )
            atom_pair_scores_list.append(atom_pair_scores)

        final_atom_pair_scores = torch.stack(atom_pair_scores_list).mean(dim=0)

        # 4. Transform 예측
        quaternions, translations, nuc_emb = self.transform_predictor(
            physics_x, learnable_emb, graph.atom_to_nucleotide,
            graph.sugar_x, graph.phosphate_x, graph.num_nucleotides
        )

        # 5. 좌표 계산
        coords = graph.apply_transforms(quaternions, translations)

        # 6. 물리 엔진 평가
        physics_eval = self.physics.evaluate(
            coords=coords,
            vdw_radii=graph.vdw_radii,
            is_donor=graph.is_donor,
            is_acceptor=graph.is_acceptor,
            nuc_ids=graph.atom_to_nucleotide,
        )

        return {
            'coords': coords,
            'quaternions': quaternions,
            'translations': translations,
            'physics_x': physics_x,
            'structural_x': structural_x,
            'learnable_emb': learnable_emb,
            'atom_pair_scores': final_atom_pair_scores,
            'physics_mask': physics_mask,  # 어떤 쌍이 attention 대상이었는지
            'physics': physics_eval,
            'atom_to_nucleotide': graph.atom_to_nucleotide,  # 추가
            'num_nucleotides': graph.num_nucleotides,  # 추가
        }

    def compute_loss(self, outputs, target_coords=None, clash_weight=10.0, hbond_weight=1.0):
        losses = {}
        physics = outputs['physics']

        losses['clash'] = physics['clash_loss'] * clash_weight
        losses['hbond'] = -physics['hbond_energy'] * hbond_weight

        # Attention guidance (물리 마스크 내에서만)
        physics_mask = outputs['physics_mask']
        hbond_possible = (physics['hbond_strength'] > 0.1).float() * physics_mask.float()

        if hbond_possible.sum() > 0:
            losses['attn_guide'] = -torch.log(outputs['atom_pair_scores'] + 1e-8) * hbond_possible
            losses['attn_guide'] = losses['attn_guide'].sum() / (hbond_possible.sum() + 1e-8)
        else:
            losses['attn_guide'] = torch.tensor(0.0)

        if target_coords is not None:
            pred = outputs['coords'] - outputs['coords'].mean(0)
            tgt = target_coords - target_coords.mean(0)
            losses['rmsd'] = torch.sqrt(((pred - tgt) ** 2).sum(-1).mean())

        losses['total'] = sum(v for k, v in losses.items() if k != 'total' and isinstance(v, torch.Tensor))
        return losses


print("PhysicsMaskedRNAModel 정의 완료")

PhysicsMaskedRNAModel 정의 완료


## 9. 테스트

In [ ]:
"""
RNAStructurePredictor - 추론용 클래스
"""


class RNAStructurePredictor:
    """RNA 구조 예측을 위한 추론 클래스"""

    def __init__(self, model: PhysicsMaskedRNAModel, device: torch.device = None):
        self.model = model
        self.device = device or next(model.parameters()).device
        self.model.eval()

    def predict(self, sequence: str) -> dict:
        """
        RNA 서열에서 3D 구조 예측

        Args:
            sequence: RNA 서열 (예: "GCAU")

        Returns:
            dict: 예측 결과
        """
        with torch.no_grad():
            graph = HeterogeneousRNAGraph(sequence, device=self.device)
            outputs = self.model(graph)

            return {
                'sequence': sequence,
                'coords': outputs['coords'].cpu().numpy(),
                'quaternions': outputs['quaternions'].cpu().numpy(),
                'translations': outputs['translations'].cpu().numpy(),
                'atom_pair_scores': outputs['atom_pair_scores'].cpu().numpy(),
                'physics': {
                    'clash_loss': outputs['physics']['clash_loss'].item(),
                    'clash_count': outputs['physics']['clash_count'].item(),
                    'hbond_energy': outputs['physics']['hbond_energy'].item(),
                    'hbond_count': outputs['physics']['hbond_count'].item(),
                },
                'atom_info': {
                    'symbols': graph.atom_symbols,
                    'base_types': graph.base_types,
                    'nucleotide_ids': graph.atom_to_nucleotide.cpu().numpy(),
                },
                'num_nucleotides': graph.num_nucleotides,
            }


print("RNAStructurePredictor 정의 완료")


In [ ]:
def test_physics_masking():
    """
    물리 마스킹 테스트

    확인:
    1. 물리적으로 가능한 쌍만 attention 계산
    2. 불변 물리 특징 유지
    """
    print("=" * 70)
    print("  Physics Masking Test")
    print("=" * 70)

    sequence = "GCAU"
    graph = HeterogeneousRNAGraph(sequence, device=device)
    model = PhysicsMaskedRNAModel(hidden_dim=64).to(device)

    print(f"\n[그래프]")
    print(f"  원자 수: {graph.num_base_atoms}")

    # Forward pass
    model.eval()
    with torch.no_grad():
        outputs = model(graph)

    # 물리 마스크 분석
    physics_mask = outputs['physics_mask']
    total_pairs = physics_mask.numel()
    valid_pairs = physics_mask.sum().item()

    print(f"\n[물리 마스킹 효과]")
    print(f"  전체 원자 쌍: {total_pairs}")
    print(f"  Attention 대상: {valid_pairs} ({100*valid_pairs/total_pairs:.1f}%)")
    print(f"  마스킹됨: {total_pairs - valid_pairs} ({100*(total_pairs-valid_pairs)/total_pairs:.1f}%)")

    # 어떤 쌍이 attention 대상인지 분석
    is_donor = graph.is_donor
    is_acceptor = graph.is_acceptor
    is_aromatic = graph.structural_x[:, 1]

    print(f"\n[원자 타입별]")
    print(f"  H-bond donor 원자: {(is_donor > 0).sum().item()}개")
    print(f"  H-bond acceptor 원자: {(is_acceptor > 0).sum().item()}개")
    print(f"  Aromatic 원자: {(is_aromatic > 0).sum().item()}개")

    # 불변성 검증
    physics_before = graph.physics_x.clone()
    physics_after = outputs['physics_x']
    is_invariant = torch.equal(physics_before, physics_after)
    print(f"\n[불변성 검증]")
    print(f"  물리 특징 불변: {is_invariant} ✅" if is_invariant else "  물리 특징 변경됨 ❌")

    return model, graph, outputs


model, graph, outputs = test_physics_masking()

  Physics Masking Test

[그래프]
  원자 수: 37

[물리 마스킹 효과]
  전체 원자 쌍: 1369
  Attention 대상: 778 (56.8%)
  마스킹됨: 591 (43.2%)

[원자 타입별]
  H-bond donor 원자: 9개
  H-bond acceptor 원자: 19개
  Aromatic 원자: 30개

[불변성 검증]
  물리 특징 불변: True ✅


## 11. 실제 데이터 학습 (CSV 데이터셋)

In [13]:
"""
CSV 데이터셋 로드 및 전처리
"""
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader


class RNADataset(Dataset):
    """
    sequences.csv와 labels.csv에서 RNA 데이터 로드

    sequences.csv: target_id, sequence, ...
    labels.csv: ID(target_id_residue), resname, resid, x_1~x_40, y_1~y_40, z_1~z_40
    """

    def __init__(self, sequences_path: str, labels_path: str, max_samples: int = None):
        # 데이터 로드
        self.sequences_df = pd.read_csv(sequences_path)
        self.labels_df = pd.read_csv(labels_path)

        # target_id 목록 추출
        self.target_ids = self.sequences_df['target_id'].unique().tolist()

        if max_samples:
            self.target_ids = self.target_ids[:max_samples]

        print(f"Loaded {len(self.target_ids)} RNA samples")

        # 데이터 전처리
        self.data_cache = {}
        self._preprocess()

    def _preprocess(self):
        """데이터 전처리 및 캐싱"""
        for target_id in self.target_ids:
            # 서열 가져오기
            seq_row = self.sequences_df[self.sequences_df['target_id'] == target_id]
            if len(seq_row) == 0:
                continue

            sequence = seq_row['sequence'].values[0]

            # 해당 target의 라벨 가져오기
            label_rows = self.labels_df[self.labels_df['ID'].str.startswith(f'{target_id}_')]
            label_rows = label_rows.sort_values('resid')

            if len(label_rows) == 0:
                continue

            # 3D 좌표 추출 (뉴클레오타이드당 대표 좌표 = 첫 번째 유효 좌표)
            coords = []
            for _, row in label_rows.iterrows():
                # 각 뉴클레오타이드의 대표 좌표 (첫 번째 유효 좌표 또는 평균)
                atom_coords = []
                for i in range(1, 41):  # x_1 ~ x_40
                    x = row.get(f'x_{i}', -1e18)
                    y = row.get(f'y_{i}', -1e18)
                    z = row.get(f'z_{i}', -1e18)

                    if x > -1e17 and y > -1e17 and z > -1e17:  # 유효한 좌표
                        atom_coords.append([x, y, z])

                if atom_coords:
                    # 평균 좌표 (centroid)
                    centroid = np.mean(atom_coords, axis=0)
                    coords.append(centroid)
                else:
                    coords.append([0.0, 0.0, 0.0])  # fallback

            coords = np.array(coords, dtype=np.float32)

            # 서열 길이와 좌표 개수 맞추기
            min_len = min(len(sequence), len(coords))
            sequence = sequence[:min_len]
            coords = coords[:min_len]

            self.data_cache[target_id] = {
                'sequence': sequence,
                'coords': coords,  # [num_nuc, 3]
            }

        # 유효한 데이터만 필터링
        self.target_ids = [tid for tid in self.target_ids if tid in self.data_cache]
        print(f"Valid samples after preprocessing: {len(self.target_ids)}")

    def __len__(self):
        return len(self.target_ids)

    def __getitem__(self, idx):
        target_id = self.target_ids[idx]
        data = self.data_cache[target_id]

        return {
            'target_id': target_id,
            'sequence': data['sequence'],
            'coords': torch.tensor(data['coords'], dtype=torch.float32),
        }


print("RNADataset 정의 완료")

RNADataset 정의 완료


In [14]:
"""
데이터 로드 및 확인
"""

# 데이터 경로 설정 (Colab이나 로컬에 맞게 수정)
SEQUENCES_PATH = 'sequences.csv'
LABELS_PATH = 'labels.csv'

# 10개 샘플만 로드
dataset = RNADataset(SEQUENCES_PATH, LABELS_PATH, max_samples=10)

# 데이터 확인
print("\n[데이터셋 샘플 확인]")
for i in range(min(3, len(dataset))):
    sample = dataset[i]
    print(f"\n{i+1}. {sample['target_id']}")
    print(f"   서열: {sample['sequence'][:30]}... (길이: {len(sample['sequence'])})")
    print(f"   좌표: {sample['coords'].shape}")

Loaded 10 RNA samples
Valid samples after preprocessing: 10

[데이터셋 샘플 확인]

1. R1107
   서열: GGGGGCCACAGCAGAAGCGUUCACGUCGCA... (길이: 69)
   좌표: torch.Size([69, 3])

2. R1108
   서열: GGGGGCCACAGCAGAAGCGUUCACGUCGCG... (길이: 69)
   좌표: torch.Size([69, 3])

3. R1116
   서열: CGCCCGGAUAGCUCAGUCGGUAGAGCAGCG... (길이: 157)
   좌표: torch.Size([157, 3])


In [ ]:
"""
RMSD 손실 함수 (실제 데이터 학습용)
"""


def compute_rmsd_loss(pred_coords: torch.Tensor, target_coords: torch.Tensor) -> torch.Tensor:
    """
    예측 좌표와 타겟 좌표 간의 RMSD 계산

    Args:
        pred_coords: [N, 3] 예측된 뉴클레오타이드 중심 좌표
        target_coords: [N, 3] 실제 뉴클레오타이드 중심 좌표

    Returns:
        RMSD 손실
    """
    # 중심 정렬 (centroid alignment)
    pred_center = pred_coords.mean(dim=0, keepdim=True)
    target_center = target_coords.mean(dim=0, keepdim=True)

    pred_centered = pred_coords - pred_center
    target_centered = target_coords - target_center

    # RMSD
    diff = pred_centered - target_centered
    rmsd = torch.sqrt((diff ** 2).sum(dim=-1).mean())

    return rmsd


def compute_supervised_loss(
    outputs: dict,
    target_nuc_coords: torch.Tensor,
    rmsd_weight: float = 1.0,
    clash_weight: float = 0.1,
    hbond_weight: float = 0.05,
) -> dict:
    """
    지도 학습 손실 함수

    Loss = RMSD + Physics Loss
    """
    # 예측된 원자 좌표에서 뉴클레오타이드 중심 계산
    pred_coords = outputs['coords']  # [num_atoms, 3]
    atom_to_nuc = outputs['atom_to_nucleotide']  # [num_atoms]

    # 뉴클레오타이드별 중심 좌표 계산
    num_nuc = outputs['num_nucleotides']
    pred_nuc_coords = []
    for nuc_id in range(num_nuc):
        mask = atom_to_nuc == nuc_id
        if mask.sum() > 0:
            nuc_center = pred_coords[mask].mean(dim=0)
            pred_nuc_coords.append(nuc_center)

    pred_nuc_coords = torch.stack(pred_nuc_coords)  # [num_nuc, 3]

    # 길이 맞추기
    min_len = min(len(pred_nuc_coords), len(target_nuc_coords))
    pred_nuc_coords = pred_nuc_coords[:min_len]
    target_nuc_coords = target_nuc_coords[:min_len]

    # 1. RMSD Loss
    rmsd_loss = compute_rmsd_loss(pred_nuc_coords, target_nuc_coords)

    # 2. Physics Loss (원자 수준)
    physics = outputs['physics']
    clash_loss = physics['clash_loss']
    hbond_loss = -physics['hbond_energy'] * 0.01  # H-bond가 많을수록 좋음

    # Total Loss
    total_loss = (
        rmsd_weight * rmsd_loss +
        clash_weight * clash_loss +
        hbond_weight * hbond_loss
    )

    return {
        'total': total_loss,
        'rmsd': rmsd_loss,
        'clash': clash_loss,
        'hbond': hbond_loss,
    }


print("Supervised loss 정의 완료")

Supervised loss 정의 완료


In [ ]:
"""
실제 데이터 학습 함수
"""


def train_with_real_data(
    model: PhysicsMaskedRNAModel,
    dataset: RNADataset,
    num_epochs: int = 10,
    lr: float = 1e-3,
    device: str = 'cpu',
):
    """
    실제 RNA 데이터로 학습
    """
    model = model.to(device)
    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    history = {
        'epoch': [],
        'total_loss': [],
        'rmsd': [],
        'clash': [],
        'hbond': [],
    }

    print("=" * 70)
    print("  RNA 3D Structure Training")
    print("=" * 70)
    print(f"  Samples: {len(dataset)}, Epochs: {num_epochs}, LR: {lr}")
    print("=" * 70)

    for epoch in range(num_epochs):
        epoch_losses = {'total': 0, 'rmsd': 0, 'clash': 0, 'hbond': 0}
        valid_samples = 0

        for idx in range(len(dataset)):
            sample = dataset[idx]
            sequence = sample['sequence']
            target_coords = sample['coords'].to(device)

            try:
                # 그래프 생성 (device 지정)
                graph = HeterogeneousRNAGraph(sequence, device=device)

                # Forward
                outputs = model(graph)

                # Loss 계산
                losses = compute_supervised_loss(outputs, target_coords)

                # Backward
                optimizer.zero_grad()
                losses['total'].backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                # 기록
                epoch_losses['total'] += losses['total'].item()
                epoch_losses['rmsd'] += losses['rmsd'].item()
                epoch_losses['clash'] += losses['clash'].item()
                epoch_losses['hbond'] += losses['hbond'].item()
                valid_samples += 1

            except Exception as e:
                print(f"  Skip {sample['target_id']}: {e}")
                continue

        if valid_samples > 0:
            # 평균 계산
            for k in epoch_losses:
                epoch_losses[k] /= valid_samples

            history['epoch'].append(epoch + 1)
            history['total_loss'].append(epoch_losses['total'])
            history['rmsd'].append(epoch_losses['rmsd'])
            history['clash'].append(epoch_losses['clash'])
            history['hbond'].append(epoch_losses['hbond'])

            print(f"Epoch {epoch+1:3d} | Loss: {epoch_losses['total']:.4f} | "
                  f"RMSD: {epoch_losses['rmsd']:.2f}Å | "
                  f"Clash: {epoch_losses['clash']:.2f} | "
                  f"H-bond: {epoch_losses['hbond']:.4f}")

    print("=" * 70)
    print("Training Complete!")

    return history


print("train_with_real_data 정의 완료")

train_with_real_data 정의 완료


In [1]:
"""
10개 샘플로 학습 실행
"""

# 모델 생성
model = PhysicsMaskedRNAModel(hidden_dim=64)

# 학습 실행
history = train_with_real_data(
    model=model,
    dataset=dataset,
    num_epochs=20,
    lr=1e-3,
)

NameError: name 'PhysicsMaskedRNAModel' is not defined

In [ ]:
"""
학습 결과 시각화
"""

def plot_training_history(history: dict):
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # 1. Total Loss
    ax = axes[0, 0]
    ax.plot(history['epoch'], history['total_loss'], 'b-', linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Total Loss')
    ax.set_title('Total Loss')
    ax.grid(True)

    # 2. RMSD
    ax = axes[0, 1]
    ax.plot(history['epoch'], history['rmsd'], 'g-', linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('RMSD (Å)')
    ax.set_title('RMSD')
    ax.grid(True)

    # 3. Clash Loss
    ax = axes[1, 0]
    ax.plot(history['epoch'], history['clash'], 'r-', linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Clash Loss')
    ax.set_title('Clash Loss')
    ax.grid(True)

    # 4. H-bond Loss
    ax = axes[1, 1]
    ax.plot(history['epoch'], history['hbond'], 'm-', linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('H-bond Loss')
    ax.set_title('H-bond Loss')
    ax.grid(True)

    plt.tight_layout()
    plt.show()


plot_training_history(history)

In [ ]:
"""
학습된 모델로 예측 및 비교
"""

def evaluate_and_visualize(model, dataset, sample_idx: int = 0):
    """
    학습된 모델로 예측하고 실제 좌표와 비교
    """
    model.eval()
    sample = dataset[sample_idx]

    print(f"\n[평가] {sample['target_id']}")
    print(f"서열: {sample['sequence'][:50]}...")

    # 예측
    predictor = RNAStructurePredictor(model)
    result = predictor.predict(sample['sequence'])

    # 타겟 좌표
    target_coords = sample['coords'].numpy()

    # 예측된 뉴클레오타이드 중심 계산
    pred_coords = result['coords']
    nuc_ids = result['atom_info']['nucleotide_ids']

    pred_nuc_coords = []
    for nuc_id in range(result['num_nucleotides']):
        mask = nuc_ids == nuc_id
        if mask.sum() > 0:
            pred_nuc_coords.append(pred_coords[mask].mean(axis=0))
    pred_nuc_coords = np.array(pred_nuc_coords)

    # 길이 맞추기
    min_len = min(len(pred_nuc_coords), len(target_coords))
    pred_nuc_coords = pred_nuc_coords[:min_len]
    target_coords = target_coords[:min_len]

    # RMSD 계산
    diff = pred_nuc_coords - target_coords
    rmsd = np.sqrt((diff ** 2).sum(axis=-1).mean())
    print(f"RMSD: {rmsd:.2f} Å")

    # 시각화
    fig = plt.figure(figsize=(14, 6))

    # 1. 예측 구조
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    ax1.scatter(pred_nuc_coords[:, 0], pred_nuc_coords[:, 1], pred_nuc_coords[:, 2],
               c=range(len(pred_nuc_coords)), cmap='viridis', s=100)
    # 연결선
    for i in range(len(pred_nuc_coords) - 1):
        ax1.plot3D([pred_nuc_coords[i, 0], pred_nuc_coords[i+1, 0]],
                   [pred_nuc_coords[i, 1], pred_nuc_coords[i+1, 1]],
                   [pred_nuc_coords[i, 2], pred_nuc_coords[i+1, 2]], 'b-', alpha=0.5)
    ax1.set_title(f'Predicted Structure\nH-bonds: {result["physics"]["hbond_count"]:.0f}')
    ax1.set_xlabel('X (Å)')
    ax1.set_ylabel('Y (Å)')
    ax1.set_zlabel('Z (Å)')

    # 2. 실제 구조
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    ax2.scatter(target_coords[:, 0], target_coords[:, 1], target_coords[:, 2],
               c=range(len(target_coords)), cmap='viridis', s=100)
    # 연결선
    for i in range(len(target_coords) - 1):
        ax2.plot3D([target_coords[i, 0], target_coords[i+1, 0]],
                   [target_coords[i, 1], target_coords[i+1, 1]],
                   [target_coords[i, 2], target_coords[i+1, 2]], 'r-', alpha=0.5)
    ax2.set_title('Ground Truth Structure')
    ax2.set_xlabel('X (Å)')
    ax2.set_ylabel('Y (Å)')
    ax2.set_zlabel('Z (Å)')

    plt.suptitle(f'{sample["target_id"]} - RMSD: {rmsd:.2f} Å', fontsize=14)
    plt.tight_layout()
    plt.show()

    return rmsd


# 첫 번째 샘플로 평가
if len(dataset) > 0:
    evaluate_and_visualize(model, dataset, sample_idx=0)

In [ ]:
"""
모든 샘플에 대해 RMSD 평가
"""

def evaluate_all_samples(model, dataset):
    model.eval()
    predictor = RNAStructurePredictor(model)

    results = []

    print("\n[전체 샘플 평가]")
    print("-" * 60)

    for idx in range(len(dataset)):
        sample = dataset[idx]

        try:
            result = predictor.predict(sample['sequence'])
            target_coords = sample['coords'].numpy()

            # 뉴클레오타이드 중심 계산
            pred_coords = result['coords']
            nuc_ids = result['atom_info']['nucleotide_ids']

            pred_nuc_coords = []
            for nuc_id in range(result['num_nucleotides']):
                mask = nuc_ids == nuc_id
                if mask.sum() > 0:
                    pred_nuc_coords.append(pred_coords[mask].mean(axis=0))
            pred_nuc_coords = np.array(pred_nuc_coords)

            # RMSD
            min_len = min(len(pred_nuc_coords), len(target_coords))
            diff = pred_nuc_coords[:min_len] - target_coords[:min_len]
            rmsd = np.sqrt((diff ** 2).sum(axis=-1).mean())

            results.append({
                'id': sample['target_id'],
                'rmsd': rmsd,
                'hbond': result['physics']['hbond_count'],
                'clash': result['physics']['clash_count'],
            })

            print(f"{sample['target_id']:<15} RMSD: {rmsd:>8.2f} Å  "
                  f"H-bonds: {result['physics']['hbond_count']:>4.0f}  "
                  f"Clashes: {result['physics']['clash_count']:>4.0f}")

        except Exception as e:
            print(f"{sample['target_id']:<15} Error: {e}")

    print("-" * 60)

    if results:
        avg_rmsd = np.mean([r['rmsd'] for r in results])
        print(f"Average RMSD: {avg_rmsd:.2f} Å")

    return results


eval_results = evaluate_all_samples(model, dataset)

In [ ]:
"""
모델 저장/로드
"""

# 모델 저장
# torch.save(model.state_dict(), 'rna_model_trained.pt')
# print("모델 저장 완료: rna_model_trained.pt")

# 모델 로드
# model = PhysicsMaskedRNAModel(hidden_dim=64)
# model.load_state_dict(torch.load('rna_model_trained.pt'))
# print("모델 로드 완료")

## 12. 요약

### 핵심 최적화: 물리 마스킹

```
물리적으로 불가능한 쌍 → Attention에서 제외!

┌─────────────────────────────────────────────────────────────┐
│                    Attention 대상 (마스킹 X)                │
├─────────────────────────────────────────────────────────────┤
│  H-bond 가능: donor ↔ acceptor                             │
│  Stacking 가능: aromatic ↔ aromatic                        │
├─────────────────────────────────────────────────────────────┤
│                    마스킹됨 (Attention = 0)                 │
├─────────────────────────────────────────────────────────────┤
│  C ↔ C (H-bond 불가, stacking 불가)                        │
│  같은 뉴클레오타이드 내 원자                                 │
└─────────────────────────────────────────────────────────────┘
```

### 효과

```
1. 계산 효율: O(N²) → O(가능한 쌍만)
2. 물리적 의미: 실제 상호작용 가능한 쌍에만 집중
3. 노이즈 감소: 불필요한 attention이 학습을 방해하지 않음
```

### 핵심 코드

```python
# 물리 마스크 계산
hbond_mask = donor_i * acceptor_j + acceptor_i * donor_j
stacking_mask = aromatic_i * aromatic_j
physics_mask = (hbond_mask | stacking_mask) & diff_nuc

# Attention에서 불가능한 쌍 제외
attn_scores = attn_scores.masked_fill(~physics_mask, -inf)
```

### 불변 특징 vs 학습 임베딩

```
┌──────────────────┬──────────────────────────────────────┐
│ 불변 물리 (10d)  │ 학습 임베딩 (hidden)                 │
├──────────────────┼──────────────────────────────────────┤
│ 원자번호, VdW    │ 문맥 정보, 상호작용 패턴             │
│ donor/acceptor   │                                      │
├──────────────────┼──────────────────────────────────────┤
│ 학습 ❌          │ 학습 ✅                              │
│ 마스크 계산용    │ GNN/Attention으로 업데이트           │
│ 물리 엔진용      │ R, T 예측용                          │
└──────────────────┴──────────────────────────────────────┘
```